In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2
from creation_functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
df_P = pd.io.parsers.read_csv(filepath_or_buffer = '7_25_filt_closed_only.csv',index_col = 0)
print df_P[:5]

                     o3_ppb      UnixTime       e2v03         no2       Temp  \
2014-07-25 20:03:00      37  1.406319e+09  380.076923  626.384615  30.700000   
2014-07-25 20:04:00      37  1.406319e+09  378.923077  624.384615  30.692308   
2014-07-25 21:06:00      39  1.406322e+09  357.916667  594.916667  28.316667   
2014-07-25 21:07:00      36  1.406322e+09  357.692308  595.000000  28.384615   
2014-07-25 21:08:00      38  1.406323e+09  357.083333  594.250000  28.400000   

                            Rh  days from start  ref_o3_smooth  
2014-07-25 20:03:00  30.369231                0            NaN  
2014-07-25 20:04:00  29.900000                0            NaN  
2014-07-25 21:06:00  32.075000                0            NaN  
2014-07-25 21:07:00  31.838462                0            NaN  
2014-07-25 21:08:00  31.608333                0            NaN  


In [27]:
df_P.drop(df_P.columns[[7]], axis=1, inplace=True)
print df_P[:5]

                     o3_ppb      UnixTime       e2v03         no2       Temp  \
2014-07-25 20:03:00      37  1.406319e+09  380.076923  626.384615  30.700000   
2014-07-25 20:04:00      37  1.406319e+09  378.923077  624.384615  30.692308   
2014-07-25 21:06:00      39  1.406322e+09  357.916667  594.916667  28.316667   
2014-07-25 21:07:00      36  1.406322e+09  357.692308  595.000000  28.384615   
2014-07-25 21:08:00      38  1.406323e+09  357.083333  594.250000  28.400000   

                            Rh  days from start  
2014-07-25 20:03:00  30.369231                0  
2014-07-25 20:04:00  29.900000                0  
2014-07-25 21:06:00  32.075000                0  
2014-07-25 21:07:00  31.838462                0  
2014-07-25 21:08:00  31.608333                0  


In [28]:
#ozone terms
df_P['O3_sqrt'] = np.sqrt(df_P['e2v03'].astype(float))
df_P['O3_sq'] = df_P['e2v03']**2
df_P['O3_cu'] = df_P['e2v03']**3
df_P['ln_O3'] = np.log(df_P['e2v03'])

#temp terms
df_P['temp_sqrt'] = np.sqrt(df_P['Temp'].astype(float))
df_P['temp_sq'] = df_P['Temp']**2
df_P['temp_cu'] = df_P['Temp']**3
df_P['ln_temp'] =np.log(df_P['Temp'])

#rh terms
df_P['rh_sqrt'] = np.sqrt(df_P['Rh'].astype(float))
df_P['rh_sq'] = df_P['Rh']**2
df_P['rh_cu'] = df_P['Rh']**3
df_P['ln_rh'] = np.log(df_P['Rh'])

#ozone/temp interactions
df_P['03_mult_temp'] = df_P['e2v03']*df_P['Temp']
df_P['03_sq_mult_temp_sq'] = ((df_P['e2v03'])**2)*df_P['Temp']**2
df_P['ln_03_mult_ln_temp'] = np.log(df_P['Temp'])*np.log(df_P['e2v03'])

#ozone/rh interactions
df_P['03_mult_rh'] = df_P['e2v03']*df_P['Rh']
df_P['03_sq_mult_rh_sq'] = (df_P['e2v03']**2)*df_P['Rh']**2
df_P['ln_03_mult_ln_rh'] = np.log(df_P['Rh'])*np.log(df_P['e2v03'])

#temp/rh interactions
df_P['temp_mult_rh'] = df_P['Temp']*df_P['Rh']
df_P['temp_sq_mult_rh_sq'] = (df_P['Temp']**2)*df_P['Rh']**2
df_P['ln_temp_mult_ln_rh'] = np.log(df_P['Temp'])*np.log(df_P['Rh'])

#ozone/rh/temp interactions
df_P['03_mult_rh_&_temp'] = df_P['e2v03']*df_P['Rh']*df_P['Temp']
df_P['03_sq_mult_rh_sq_&_temp_sq'] = (df_P['e2v03']**2)*(df_P['Rh']**2)*df_P['Temp']**2
df_P['ln_03_mult_ln_rh_&_ln_temp'] = np.log(df_P['Temp'])*np.log(df_P['Rh'])*np.log(df_P['e2v03'])

feature_list = ['e2v03', 'ln_O3', 'Temp', 'ln_temp', 'Rh', 'ln_rh']

for i in range(0,len(feature_list)):
    make_func_caller_find_lag_integral(df_P, 1, 5, 1, feature_list[i])
    make_func_caller_find_lead_integral(df_P, 1, 5, 1, feature_list[i])
    make_func_caller_find_lag_slope(df_P, 1, 5, 1, feature_list[i])
    make_func_caller_find_lead_slope(df_P, 1, 5, 1, feature_list[i])
    make_func_caller_find_lag_integral(df_P, 1, 5, 1, feature_list[i])
    make_func_caller_find_lead_integral(df_P, 1, 5, 1, feature_list[i])
    make_func_caller_find_lag_slope(df_P, 1, 5, 1, feature_list[i])
    make_func_caller_find_lead_slope(df_P, 1, 5, 1, feature_list[i])

features = list(df_P.columns)
print features

['o3_ppb', 'UnixTime', 'e2v03', 'no2', 'Temp', 'Rh', 'days from start', 'O3_sqrt', 'O3_sq', 'O3_cu', 'ln_O3', 'temp_sqrt', 'temp_sq', 'temp_cu', 'ln_temp', 'rh_sqrt', 'rh_sq', 'rh_cu', 'ln_rh', '03_mult_temp', '03_sq_mult_temp_sq', 'ln_03_mult_ln_temp', '03_mult_rh', '03_sq_mult_rh_sq', 'ln_03_mult_ln_rh', 'temp_mult_rh', 'temp_sq_mult_rh_sq', 'ln_temp_mult_ln_rh', '03_mult_rh_&_temp', '03_sq_mult_rh_sq_&_temp_sq', 'ln_03_mult_ln_rh_&_ln_temp', 'e2v03_int_lag_1', 'e2v03_int_lag_2', 'e2v03_int_lag_3', 'e2v03_int_lag_4', 'e2v03_int_lag_5', 'e2v03_int_lead_1', 'e2v03_int_lead_2', 'e2v03_int_lead_3', 'e2v03_int_lead_4', 'e2v03_int_lead_5', 'diff', 'e2v03_slope_lag_1', 'e2v03_slope_lag_2', 'e2v03_slope_lag_3', 'e2v03_slope_lag_4', 'e2v03_slope_lag_5', 'e2v03_slope_lead_1', 'e2v03_slope_lead_2', 'e2v03_slope_lead_3', 'e2v03_slope_lead_4', 'e2v03_slope_lead_5', 'ln_O3_int_lag_1', 'ln_O3_int_lag_2', 'ln_O3_int_lag_3', 'ln_O3_int_lag_4', 'ln_O3_int_lag_5', 'ln_O3_int_lead_1', 'ln_O3_int_lead_2'

In [29]:
#Delete the first and last 90 rows of the dataframe to remove all NaNs.
df_P = df_P.ix[90:len(df_P['e2v03'])-90]

In [15]:
#ozone/temp
df_P['o3_temp_int_lag_30'] = df_P['e2v03_int_lag_30'] * df_P['Temp_int_lag_30'] 
df_P['o3_temp_slope_lag_30'] = df_P['e2v03_slope_lag_30'] * df_P['Temp_slope_lag_30']
df_P['o3_temp_int_lag_60'] = df_P['e2v03_int_lag_60'] * df_P['Temp_int_lag_60'] 
df_P['o3_temp_slope_lag_60'] = df_P['e2v03_slope_lag_60'] * df_P['Temp_slope_lag_60'] 
df_P['o3_temp_int_lag_90'] = df_P['e2v03_int_lag_90'] * df_P['Temp_int_lag_90'] 
df_P['o3_temp_slope_lag_90'] = df_P['e2v03_slope_lag_90'] * df_P['Temp_slope_lag_90']
df_P['o3_temp_int_lag_5'] = df_P['e2v03_int_lag_5'] * df_P['Temp_int_lag_5'] 
df_P['o3_temp_slope_lag_5'] = df_P['e2v03_slope_lag_5'] * df_P['Temp_slope_lag_5']

df_P['ln_o3_temp_int_lag_30'] = df_P['ln_O3_int_lag_30'] * df_P['ln_temp_int_lag_30'] 
df_P['ln_o3_temp_slope_lag_30'] = df_P['ln_O3_slope_lag_30'] * df_P['ln_temp_slope_lag_30']
df_P['ln_o3_temp_int_lag_60'] = df_P['ln_O3_int_lag_60'] * df_P['ln_temp_int_lag_60'] 
df_P['ln_o3_temp_slope_lag_60'] = df_P['ln_O3_slope_lag_60'] * df_P['ln_temp_slope_lag_60'] 
df_P['ln_o3_temp_int_lag_90'] = df_P['ln_O3_int_lag_90'] * df_P['ln_temp_int_lag_90'] 
df_P['ln_o3_temp_slope_lag_90'] = df_P['ln_O3_slope_lag_90'] * df_P['ln_temp_slope_lag_90']
df_P['ln_o3_temp_int_lag_5'] = df_P['ln_O3_int_lag_5'] * df_P['ln_temp_int_lag_5'] 
df_P['ln_o3_temp_slope_lag_5'] = df_P['ln_O3_slope_lag_5'] * df_P['ln_temp_slope_lag_5']

df_P['o3_temp_int_lead_30'] = df_P['e2v03_int_lead_30'] * df_P['Temp_int_lead_30'] 
df_P['o3_temp_slope_lead_30'] = df_P['e2v03_slope_lead_30'] * df_P['Temp_slope_lead_30'] 
df_P['o3_temp_int_lead_60'] = df_P['e2v03_int_lead_60'] * df_P['Temp_int_lead_60'] 
df_P['o3_temp_slope_lead_60'] = df_P['e2v03_slope_lead_60'] * df_P['Temp_slope_lead_60'] 
df_P['o3_temp_int_lead_90'] = df_P['e2v03_int_lead_90'] * df_P['Temp_int_lead_90'] 
df_P['o3_temp_slope_lead_90'] = df_P['e2v03_slope_lead_90'] * df_P['Temp_slope_lead_90']
df_P['o3_temp_int_lead_5'] = df_P['e2v03_int_lead_5'] * df_P['Temp_int_lead_5'] 
df_P['o3_temp_slope_lead_5'] = df_P['e2v03_slope_lead_5'] * df_P['Temp_slope_lead_5']

df_P['ln_o3_temp_int_lead_30'] = df_P['ln_O3_int_lead_30'] * df_P['ln_temp_int_lead_30'] 
df_P['ln_o3_temp_slope_lead_30'] = df_P['ln_O3_slope_lead_30'] * df_P['ln_temp_slope_lead_30'] 
df_P['ln_o3_temp_int_lead_60'] = df_P['ln_O3_int_lead_60'] * df_P['ln_temp_int_lead_60'] 
df_P['ln_o3_temp_slope_lead_60'] = df_P['ln_O3_slope_lead_60'] * df_P['ln_temp_slope_lead_60'] 
df_P['ln_o3_temp_int_lead_90'] = df_P['ln_O3_int_lead_90'] * df_P['ln_temp_int_lead_90'] 
df_P['ln_o3_temp_slope_lead_90'] = df_P['ln_O3_slope_lead_90'] * df_P['ln_temp_slope_lead_90']
df_P['ln_o3_temp_int_lead_5'] = df_P['ln_O3_int_lead_5'] * df_P['ln_temp_int_lead_5'] 
df_P['ln_o3_temp_slope_lead_5'] = df_P['ln_O3_slope_lead_5'] * df_P['ln_temp_slope_lead_5']

#temp/rh
df_P['temp_rh_int_lag_30'] = df_P['Temp_int_lag_30'] * df_P['Rh_int_lag_30'] 
df_P['temp_rh_slope_lag_30'] = df_P['Temp_slope_lag_30'] * df_P['Rh_slope_lag_30'] 
df_P['temp_rh_int_lag_60'] = df_P['Temp_int_lag_60'] * df_P['Rh_int_lag_60'] 
df_P['temp_rh_slope_lag_60'] = df_P['Temp_slope_lag_60'] * df_P['Rh_slope_lag_60'] 
df_P['temp_rh_int_lag_90'] = df_P['Temp_int_lag_90'] * df_P['Rh_int_lag_90'] 
df_P['temp_rh_slope_lag_90'] = df_P['Temp_slope_lag_90'] * df_P['Rh_slope_lag_90'] 
df_P['temp_rh_int_lag_5'] = df_P['Temp_int_lag_5'] * df_P['Rh_int_lag_5'] 
df_P['temp_rh_slope_lag_5'] = df_P['Temp_slope_lag_5'] * df_P['Rh_slope_lag_5']

df_P['ln_temp_rh_int_lag_30'] = df_P['ln_temp_int_lag_30'] * df_P['ln_rh_int_lag_30'] 
df_P['ln_temp_rh_slope_lag_30'] = df_P['ln_temp_slope_lag_30'] * df_P['ln_rh_slope_lag_30'] 
df_P['ln_temp_rh_int_lag_60'] = df_P['ln_temp_int_lag_60'] * df_P['ln_rh_int_lag_60'] 
df_P['ln_temp_rh_slope_lag_60'] = df_P['ln_temp_slope_lag_60'] * df_P['ln_rh_slope_lag_60'] 
df_P['ln_temp_rh_int_lag_90'] = df_P['ln_temp_int_lag_90'] * df_P['ln_rh_int_lag_90'] 
df_P['ln_temp_rh_slope_lag_90'] = df_P['ln_temp_slope_lag_90'] * df_P['ln_rh_slope_lag_90'] 
df_P['ln_temp_rh_int_lag_5'] = df_P['ln_temp_int_lag_5'] * df_P['ln_rh_int_lag_5'] 
df_P['ln_temp_rh_slope_lag_5'] = df_P['ln_temp_slope_lag_5'] * df_P['ln_rh_slope_lag_5']

df_P['temp_rh_int_lead_30'] = df_P['Temp_int_lead_30'] * df_P['Rh_int_lead_30'] 
df_P['temp_rh_slope_lead_30'] = df_P['Temp_slope_lead_30'] * df_P['Rh_slope_lead_30'] 
df_P['temp_rh_int_lead_60'] = df_P['Temp_int_lead_60'] * df_P['Rh_int_lead_60'] 
df_P['temp_rh_slope_lead_60'] = df_P['Temp_slope_lead_60'] * df_P['Rh_slope_lead_60'] 
df_P['temp_rh_int_lead_90'] = df_P['Temp_int_lead_90'] * df_P['Rh_int_lead_90'] 
df_P['temp_rh_slope_lead_90'] = df_P['Temp_slope_lead_90'] * df_P['Rh_slope_lead_90']
df_P['temp_rh_int_lead_5'] = df_P['Temp_int_lead_5'] * df_P['Rh_int_lead_5'] 
df_P['temp_rh_slope_lead_5'] = df_P['Temp_slope_lead_5'] * df_P['Rh_slope_lead_5']

df_P['ln_temp_rh_int_lead_30'] = df_P['ln_temp_int_lead_30'] * df_P['ln_rh_int_lead_30'] 
df_P['ln_temp_rh_slope_lead_30'] = df_P['ln_temp_slope_lead_30'] * df_P['ln_rh_slope_lead_30'] 
df_P['ln_temp_rh_int_lead_60'] = df_P['ln_temp_int_lead_60'] * df_P['ln_rh_int_lead_60'] 
df_P['ln_temp_rh_slope_lead_60'] = df_P['ln_temp_slope_lead_60'] * df_P['ln_rh_slope_lead_60'] 
df_P['ln_temp_rh_int_lead_90'] = df_P['ln_temp_int_lead_90'] * df_P['ln_rh_int_lead_90'] 
df_P['ln_temp_rh_slope_lead_5'] = df_P['ln_temp_slope_lead_5'] * df_P['ln_rh_slope_lead_5']
df_P['ln_temp_rh_int_lead_5'] = df_P['ln_temp_slope_lead_5'] * df_P['ln_rh_slope_lead_5']

#ozone/rh
df_P['o3_rh_int_lag_30'] = df_P['e2v03_int_lag_30'] * df_P['Rh_int_lag_30'] 
df_P['o3_rh_slope_lag_30'] = df_P['e2v03_slope_lag_30'] * df_P['Rh_slope_lag_30'] 
df_P['o3_rh_int_lag_60'] = df_P['e2v03_int_lag_60'] * df_P['Rh_int_lag_60'] 
df_P['o3_rh_slope_lag_60'] = df_P['e2v03_slope_lag_60'] * df_P['Rh_slope_lag_60'] 
df_P['o3_rh_int_lag_90'] = df_P['e2v03_int_lag_90'] * df_P['Rh_int_lag_90'] 
df_P['o3_rh_slope_lag_90'] = df_P['e2v03_slope_lag_90'] * df_P['Rh_slope_lag_90'] 
df_P['o3_rh_int_lag_5'] = df_P['e2v03_int_lag_5'] * df_P['Rh_int_lag_5'] 
df_P['o3_rh_slope_lag_5'] = df_P['e2v03_slope_lag_5'] * df_P['Rh_slope_lag_5'] 

df_P['ln_o3_rh_int_lag_30'] = df_P['ln_O3_int_lag_30'] * df_P['ln_rh_int_lag_30'] 
df_P['ln_o3_rh_slope_lag_30'] = df_P['ln_O3_slope_lag_30'] * df_P['ln_rh_slope_lag_30'] 
df_P['ln_o3_rh_int_lag_60'] = df_P['ln_O3_int_lag_60'] * df_P['ln_rh_int_lag_60'] 
df_P['ln_o3_rh_slope_lag_60'] = df_P['ln_O3_slope_lag_60'] * df_P['ln_rh_slope_lag_60'] 
df_P['ln_o3_rh_int_lag_90'] = df_P['ln_O3_int_lag_90'] * df_P['ln_rh_int_lag_90'] 
df_P['ln_o3_rh_slope_lag_90'] = df_P['ln_O3_slope_lag_90'] * df_P['ln_rh_slope_lag_90'] 
df_P['ln_o3_rh_int_lag_5'] = df_P['ln_O3_int_lag_5'] * df_P['ln_rh_int_lag_5'] 
df_P['ln_o3_rh_slope_lag_5'] = df_P['ln_O3_slope_lag_5'] * df_P['ln_rh_slope_lag_5'] 

df_P['o3_rh_int_lead_30'] = df_P['e2v03_int_lead_30'] * df_P['Rh_int_lead_30'] 
df_P['o3_rh_slope_lead_30'] = df_P['e2v03_slope_lead_30'] * df_P['Rh_slope_lead_30'] 
df_P['o3_rh_int_lead_60'] = df_P['e2v03_int_lead_60'] * df_P['Rh_int_lead_60'] 
df_P['o3_rh_slope_lead_60'] = df_P['e2v03_slope_lead_60'] * df_P['Rh_slope_lead_60'] 
df_P['o3_rh_int_lead_90'] = df_P['e2v03_int_lead_90'] * df_P['Rh_int_lead_90'] 
df_P['o3_rh_slope_lead_90'] = df_P['e2v03_slope_lead_90'] * df_P['Rh_slope_lead_90']
df_P['o3_rh_int_lead_5'] = df_P['e2v03_int_lead_5'] * df_P['Rh_int_lead_5'] 
df_P['o3_rh_slope_lead_5'] = df_P['e2v03_slope_lead_5'] * df_P['Rh_slope_lead_5']

df_P['ln_o3_rh_int_lead_30'] = df_P['ln_O3_int_lead_30'] * df_P['ln_rh_int_lead_30'] 
df_P['ln_o3_rh_slope_lead_30'] = df_P['ln_O3_slope_lead_30'] * df_P['ln_rh_slope_lead_30'] 
df_P['ln_o3_rh_int_lead_60'] = df_P['ln_O3_int_lead_60'] * df_P['ln_rh_int_lead_60'] 
df_P['ln_o3_rh_slope_lead_60'] = df_P['ln_O3_slope_lead_60'] * df_P['ln_rh_slope_lead_60'] 
df_P['ln_o3_rh_int_lead_90'] = df_P['ln_O3_int_lead_90'] * df_P['ln_rh_int_lead_90'] 
df_P['ln_o3_rh_slope_lead_90'] = df_P['ln_O3_slope_lead_90'] * df_P['ln_rh_slope_lead_90']
df_P['ln_o3_rh_int_lead_5'] = df_P['ln_O3_int_lead_5'] * df_P['ln_rh_int_lead_5'] 
df_P['ln_o3_rh_slope_lead_5'] = df_P['ln_O3_slope_lead_5'] * df_P['ln_rh_slope_lead_5']

#ozone/temp/rh
df_P['o3_temp_rh_int_lag_30'] = df_P['Temp_int_lag_30'] * df_P['Rh_int_lag_30']*df_P['e2v03_int_lag_30'] 
df_P['o3_temp_rh_slope_lag_30'] = df_P['Temp_slope_lag_30'] * df_P['Rh_slope_lag_30'] * df_P['e2v03_slope_lag_30'] 
df_P['o3_temp_rh_int_lag_60'] = df_P['Temp_int_lag_60'] * df_P['Rh_int_lag_60']*df_P['e2v03_int_lag_60'] 
df_P['o3_temp_rh_slope_lag_60'] = df_P['Temp_slope_lag_60'] * df_P['Rh_slope_lag_60'] * df_P['e2v03_slope_lag_60'] 
df_P['o3_temp_rh_int_lag_90'] = df_P['Temp_int_lag_90'] * df_P['Rh_int_lag_90'] * df_P['e2v03_int_lag_90']
df_P['o3_temp_rh_slope_lag_90'] = df_P['Temp_slope_lag_90'] * df_P['Rh_slope_lag_90'] * df_P['e2v03_slope_lag_90'] 
df_P['o3_temp_rh_int_lag_5'] = df_P['Temp_int_lag_5'] * df_P['Rh_int_lag_5'] * df_P['e2v03_int_lag_5']
df_P['o3_temp_rh_slope_lag_5'] = df_P['Temp_slope_lag_5'] * df_P['Rh_slope_lag_5'] * df_P['e2v03_slope_lag_5'] 

df_P['ln_o3_temp_rh_int_lag_30'] = df_P['ln_temp_int_lag_30'] * df_P['ln_rh_int_lag_30']*df_P['ln_O3_int_lag_30'] 
df_P['ln_o3_temp_rh_slope_lag_30'] = df_P['ln_temp_slope_lag_30'] * df_P['ln_rh_slope_lag_30'] * df_P['ln_O3_slope_lag_30'] 
df_P['ln_o3_temp_rh_int_lag_60'] = df_P['ln_temp_int_lag_60'] * df_P['ln_rh_int_lag_60']*df_P['ln_O3_int_lag_60'] 
df_P['ln_o3_temp_rh_slope_lag_60'] = df_P['ln_temp_slope_lag_60'] * df_P['ln_rh_slope_lag_60'] * df_P['ln_O3_slope_lag_60'] 
df_P['ln_o3_temp_rh_int_lag_90'] = df_P['ln_temp_int_lag_90'] * df_P['ln_rh_int_lag_90'] * df_P['ln_O3_int_lag_90']
df_P['ln_o3_temp_rh_slope_lag_90'] = df_P['ln_temp_slope_lag_90'] * df_P['ln_rh_slope_lag_90'] * df_P['ln_O3_slope_lag_90'] 
df_P['ln_o3_temp_rh_int_lag_5'] = df_P['ln_temp_int_lag_5'] * df_P['ln_rh_int_lag_5'] * df_P['ln_O3_int_lag_5']
df_P['ln_o3_temp_rh_slope_lag_5'] = df_P['ln_temp_slope_lag_5'] * df_P['ln_rh_slope_lag_5'] * df_P['ln_O3_slope_lag_5'] 

df_P['o3_temp_rh_int_lead_30'] = df_P['Temp_int_lead_30'] * df_P['Rh_int_lead_30'] * df_P['e2v03_int_lead_30'] 
df_P['o3_temp_rh_slope_lead_30'] = df_P['Temp_slope_lead_30'] * df_P['Rh_slope_lead_30'] * df_P['e2v03_slope_lead_30']
df_P['o3_temp_rh_int_lead_60'] = df_P['Temp_int_lead_60'] * df_P['Rh_int_lead_60'] * df_P['e2v03_int_lead_60'] 
df_P['o3_temp_rh_slope_lead_60'] = df_P['Temp_slope_lead_60'] * df_P['Rh_slope_lead_60'] * df_P['e2v03_slope_lead_60'] 
df_P['o3_temp_rh_int_lead_90'] = df_P['Temp_int_lead_90'] * df_P['Rh_int_lead_90'] * df_P['e2v03_int_lead_90']
df_P['o3_temp_rh_slope_lead_90'] = df_P['Temp_slope_lead_90'] * df_P['Rh_slope_lead_90'] * df_P['e2v03_slope_lead_90']
df_P['o3_temp_rh_int_lead_5'] = df_P['Temp_int_lead_5'] * df_P['Rh_int_lead_5'] * df_P['e2v03_int_lead_5']
df_P['o3_temp_rh_slope_lead_5'] = df_P['Temp_slope_lead_5'] * df_P['Rh_slope_lead_5'] * df_P['e2v03_slope_lead_5']

df_P['ln_o3_temp_rh_int_lead_30'] = df_P['ln_temp_int_lead_30'] * df_P['ln_rh_int_lead_30'] * df_P['ln_O3_int_lead_30'] 
df_P['ln_o3_temp_rh_slope_lead_30'] = df_P['ln_temp_slope_lead_30'] * df_P['ln_rh_slope_lead_30'] * df_P['ln_O3_slope_lead_30']
df_P['ln_o3_temp_rh_int_lead_60'] = df_P['ln_temp_int_lead_60'] * df_P['ln_rh_int_lead_60'] * df_P['ln_O3_int_lead_60'] 
df_P['ln_o3_temp_rh_slope_lead_60'] = df_P['ln_temp_slope_lead_60'] * df_P['ln_rh_slope_lead_60'] * df_P['ln_O3_slope_lead_60']
df_P['ln_o3_temp_rh_int_lead_90'] = df_P['ln_temp_int_lead_90'] * df_P['ln_rh_int_lead_90'] * df_P['ln_O3_int_lead_90']
df_P['ln_o3_temp_rh_slope_lead_90'] = df_P['ln_temp_slope_lead_90'] * df_P['ln_rh_slope_lead_90'] * df_P['ln_O3_slope_lead_90']
df_P['ln_o3_temp_rh_int_lead_5'] = df_P['ln_temp_int_lead_5'] * df_P['ln_rh_int_lead_5'] * df_P['ln_O3_int_lead_5']
df_P['ln_o3_temp_rh_slope_lead_5'] = df_P['ln_temp_slope_lead_5'] * df_P['ln_rh_slope_lead_5'] * df_P['ln_O3_slope_lead_5']

In [16]:
df_P['ref_o3_smooth'].plot(figsize = (15,15))

KeyError: 'ref_o3_smooth'

In [30]:
df_P.to_csv(path_or_buf = '7_25_closed_only.csv')